<a href="https://colab.research.google.com/github/vrnTEK/Deep-Learning/blob/master/Ungraded_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
_=torch.tensor([0.2126, 0.7152, 0.0722], names = ['c'])

/pytorch/c10/core/TensorImpl.h:806: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable.


In [0]:
img_t = torch.randn(3,5,5)
weights = torch.tensor([0.2126,0.7152,0.0722])

In [0]:
batch_t = torch.randn(2,3,5,5)

In [11]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [12]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights =(batch_t * unsqueezed_weights)
img_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [13]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw',img_t,weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [14]:
weights_named = torch.tensor([0.2126,0.7152,0.0722], names = ['channels'])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [15]:
img_named = img_t.refine_names(...,'channels', 'rows', 'columns')
batch_named= batch_t.refine_names(...,'channels', 'rows', 'columns')
print("img named:", img_named.shape,img_named.names)
print("batch named", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [16]:

weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape,weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [17]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [18]:
try:
    gray_named = (img_named[...,:3] * weights_named).sum('channels')
except Exception as e:
    print(e)

Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.


In [19]:

gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))